Resources:

https://www.gaussianwaves.com/2020/07/bpsk-python-matlab-bit-error-rate-simulation/

https://www.gaussianwaves.com/2015/06/how-to-generate-awgn-noise-in-matlaboctave-without-using-in-built-awgn-function/

Use pip to installed required packages:

In [ ]:
! pip install numpy matplotlib

In [27]:
import numpy as np
import matplotlib.pyplot as plt

In [28]:
def num_bits_per_symbol(M):
    """
    Compute the number of bits per symbol e.g. if M = 8, then k = 3
    """
    k = np.log2(M)
    return k

In [29]:
def compute_M_PSK_constellation(M, A):
    """
    M : Number of symbols in the constellation e.g. M=2 for BPSK
    A : Amplitude
    """
    constellation_points = []
    for m in range(0,M):
        constellation_points.append(np.array([
            A*np.cos(m/M*2.0*np.pi),  # in-phase component,   I
            A*np.sin(m/M*2.0*np.pi)   # quadrature component, Q
        ]))
    constellation = np.asarray(constellation_points)

    constellation_lookup_table = {}
    for m in range(0,M):
        print(type(m))
        constellation_lookup_table[m] = np.binary_repr(m, width=int(num_bits_per_symbol(M)))

    return constellation, constellation_lookup_table


Try the constellation computation:

In [ ]:
M4_constellation, M4_constellation_table = compute_M_PSK_constellation(4, 1)
print(M4_constellation_table)
plt.scatter(M4_constellation[:, 0], M4_constellation[:, 1])
plt.axhline(0, color='black', linewidth=.5)
plt.axvline(0, color='black', linewidth=.5)
plt.xlabel("I, in-phase")
plt.ylabel("Q, quadrature")

In [ ]:
M8_constellation, M8_constellation_table = compute_M_PSK_constellation(8, 1)
plt.scatter(M8_constellation[:, 0], M8_constellation[:, 1])
plt.axhline(0, color='black', linewidth=.5)
plt.axvline(0, color='black', linewidth=.5)
plt.xlabel("I, in-phase")
plt.ylabel("Q, quadrature")

In [32]:
def convert_binary_array_to_decimal(array_in):
    num_bits = np.shape(array_in)[-1]
    two_powers = []
    for i in range(num_bits, 0, -1):
        two_powers.append(2**(i-1))
    two_powers = np.asarray(two_powers)
    decimal = array_in.dot(two_powers)
    decimal = decimal.astype(int)
    return(decimal)


In [ ]:
print(convert_binary_array_to_decimal(np.array([1,0,1])))

In [34]:
def modulate(bit_sequence, constellation, M):
    # Divide the bit_sequence into batches each of size k
    # Map the bit batches to the constellation
    k = num_bits_per_symbol(M)
    batches = np.split(bit_sequence, np.shape(bit_sequence)[-1]/k) # risky
    print("batches = \n", batches)
    batches = np.asarray(batches)
    print("batches (as array) = \n", batches)
    decimals = convert_binary_array_to_decimal(batches)
    print("decimal = \n", decimals)
    print("constellation = \n", constellation)
    signal = constellation[decimals]
    print("signal = \n", signal)
    return signal

In [ ]:
signal = modulate(np.array([1,0,0,1,1,1]), M8_constellation, 8)

In [ ]:
# TO DO:  thnk about how to compute the power in the signal.  I think it needs to involve computing 
# the distance of each symbol from the IQ diagram origin
print(sum(signal**2/len(signal))) # too simplistic, each symbol should have equal power, as all that's different is the phase.

In [37]:
def find_nearest_constellation_point(point_in, constellation, constellation_table):
    distances = []
    for point in constellation:
        compute_distance = lambda x_in, y_in, x_fix, y_fix : np.sqrt((x_in-x_fix)**2 + (y_in-y_fix)**2)
        distance = compute_distance(point_in[0], point_in[1], point[0], point[1])
        distances.append(distance)
    print(distances)
    index_of_closest_point = np.argmin(distances)
    symbol = constellation_table[index_of_closest_point]
    return symbol

In [ ]:
print(find_nearest_constellation_point(np.array([1.0, 1.0]), M8_constellation, M8_constellation_table))

In [39]:
def add_awgn_from_channel(tx_signal, EbN0dB, k):
    gamma = 10**(EbN0dB*k/10)  # Convert EbN0 to SNR in linear scale (not in dBs), not sure if multiply by k is correct...
    signal_power = sum(abs(tx_signal)**2)/len(tx_signal) # this is wrong from general case of not BPSK, see above.
    noise_spectral_density = None
    noise_vector = None
    rx_signal = tx_signal + noise_vector
    return rx_signal

In [40]:
def demodulate(constellation):
    
    return None

In [ ]:
num_symbols = 10**5
EbN0dBs = np.arange(-4, 13, 2)

print(EbN0dBs)